In [6]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain langchain-community langchainhub langchain-chroma beautifulsoup4
!pip install -q langchain_google_genai

In [1]:
import os
os.environ["LANGSMITH_TRACING"] ="true"
os.environ["LANGSMITH_ENDPOINT"] ="https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] ="*****a"
os.environ["LANGSMITH_PROJECT"] ="RAG_With_Memory"
os.environ["GOOGLE_API_KEY"] = "A******RGSaeNIJgyg"

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
gemini_embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model = "gemini-1.5-pro-001" , convert_system_message_to_human=True)

In [9]:
print(model.invoke("What is the capital of India?").content)

The capital of India is **New Delhi**. 


In [10]:
import bs4
from langchain import hub

In [12]:
from langchain.chains import create_retrieval_chain

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [14]:
from langchain_chroma import Chroma

In [15]:
from langchain_community.document_loaders import WebBaseLoader # It will fetch the data directly from internet

In [17]:
from langchain_core.prompts import ChatPromptTemplate

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [20]:
from langchain_core.prompts import MessagesPlaceholder

In [21]:
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)

In [22]:
doc = loader.load()

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)

In [28]:
vectorstore = Chroma.from_documents(documents=splits, embedding=gemini_embeddings)
retriever = vectorstore.as_retriever()

In [29]:
retriever

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x787e2c094850>, search_kwargs={})

In [30]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

In [31]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)